In [2]:
import numpy as np
import os
import pandas as pd
import nibabel as nib

from bokeh.io import curdoc, output_notebook, output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, DataRange1d, Range1d, VBox, HBox, Select
from bokeh.palettes import Blues4
from bokeh.plotting import Figure, show
from scipy.signal import savgol_filter

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
behav_data_f = '../Phenotypic_V1_0b_preprocessed1.csv'
df = pd.read_csv(behav_data_f)

In [5]:
df = df.loc[df['func_perc_fd'].notnull(), :]
df = df.loc[df['FILE_ID']!='no_filename', :]
df['AGE_YRS'] = np.floor(df['AGE_AT_SCAN'])


In [6]:
#for col in df.columns:
#    print(col)

In [7]:
def get_dataset(src, diagnosis):
    df = src[src.DSM_IV_TR == int(diagnosis)].copy()
    df = df.set_index(['AGE_AT_SCAN'])
    df.sort_index(inplace=True)
    return ColumnDataSource(data=df)

In [32]:
def make_plot(source, title):
    plot = Figure(plot_width=600, plot_height=600, tools="", toolbar_location=None)
    plot.title.text = title
    colors = Blues4[0:3]

    plot.scatter(x='AGE_AT_SCAN', y='func_perc_fd', source=source)
    
    # fixed attributes
    plot.xaxis.axis_label = "Age at scan (y)"
    plot.yaxis.axis_label = "Percent Frame Displacement Exceeding Threshold"
    plot.axis.major_label_text_font_size = "8pt"
    plot.axis.axis_label_text_font_size = "8pt"
    plot.axis.axis_label_text_font_style = "bold"

    return plot


In [33]:
def update_plot(diagnosis, age_range):
    plot.title.text = diagnosis

    src = get_dataset(df, diagnosis)
    source.data.update(src.data)
    source.push_notebook()

In [34]:
df.DSM_IV_TR.dtype

dtype('int64')

In [35]:
diagnosis = '0'
source = get_dataset(df, diagnosis)
plot = make_plot(source, diagnosis)
controls = VBox(diagnosis_select)
layout = column(plot)

In [36]:
show(layout, notebook_handle=True)

In [31]:
from ipywidgets import interact
interact(update_plot, diagnosis=['0','1','2','3'], age_range=(6,19))

/home/ubuntu/.conda/envs/nhw16_cluster/lib/python2.7/site-packages/ipykernel/__main__.py:6: BokehDeprecationWarning: bokeh.models.sources.push_notebook was deprecated in Bokeh 0.11.0; please use bokeh.io.push_notebook instead


In [37]:
interact?

In [ ]:
for f_id in df.loc[:, 'FILE_ID'].values:
    if not f_id == "no_filename":
        print(f_id)

In [ ]:
corr_df = pd.read_table('~/DATA/test_file.1D')

plt.imshow(corr_df.corr(), interpolation='nearest')